In [1]:
import torch
import numpy as np
import pandas as pd
import plotly.express as px
from copy import deepcopy


from model import LR
from data import SyntheticDataset, FairnessDataset
from ei_model_dev import FairBatch
from ei_effort import Optimal_Effort, PGD_Effort
from ei_utils import *

In [2]:
dataset = SyntheticDataset(seed=0)

In [3]:
def append_res(d, acc, ei):
    d['accuracy'].append(acc)
    d['ei_disparity'].append(ei)
        
def get_res(d, id, hp):
    res = {'id': [], 'alpha': [], 'lambda': [], 'delta': [], 'accuracy_mean': [], 'accuracy_std': [], 'ei_disparity_mean': [], 'ei_disparity_std': []}
    res['id'].append(id)
    res['alpha'].append(hp['alpha'])
    res['lambda'].append(hp['lambda'])
    res['delta'].append(hp['delta'])
    res['accuracy_mean'].append(np.mean(d['accuracy']))
    res['accuracy_std'].append(np.std(d['accuracy']))
    res['ei_disparity_mean'].append(np.mean(d['ei_disparity']))
    res['ei_disparity_std'].append(np.std(d['ei_disparity']))
    
    return res

def get_model(models):
    weights = []
    bias = []
    for model in models:
        for module in model.layers:
            if hasattr(module, 'weight'):
                weights.append(module.weight.data)
            if hasattr(module, 'bias'):
                bias.append(module.bias.data)
            
    weights = torch.cat(weights).mean(dim=0)
    bias = torch.cat(bias).mean(dim=0)
    return weights, bias

In [4]:
def lr_fb_model_runner(dataset, hp, seeds):
    tau = 0.5
    train_metrics = {'alpha': [], 'accuracy': [], 'ei_disparity': []}
    val_metrics = deepcopy(train_metrics)
    test_metrics = deepcopy(train_metrics)
    ei_models = []
    
    if hp['optimal_effort']:
        effort = Optimal_Effort(hp['delta'])
    else:
        effort = PGD_Effort(hp['delta'])
    
    for seed in seeds:
        train_tensors, val_tensors, test_tensors = dataset.tensor(fold=seed, z_blind=hp['z_blind'])
        train_dataset = FairnessDataset(*train_tensors, dataset.imp_feats)
        val_dataset = FairnessDataset(*val_tensors, dataset.imp_feats)
        test_dataset = FairnessDataset(*test_tensors, dataset.imp_feats)
        
        model = LR(num_features=train_dataset.X.shape[1])
        ei_model = FairBatch(model, effort, tau)
        
        ei_model.train(
            train_dataset, 
            sensitive_attrs=dataset.sensitive_attrs,
            lamb=hp['lambda'],
            lr=hp['learning_rate'],
            alpha=hp['alpha']
            )

        
        Y_hat, Y_hat_max = ei_model.predict(train_dataset, hp['alpha'], dataset.sensitive_attrs)
        train_acc, train_ei = model_performance(train_dataset.Y.detach().numpy(), train_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)
        append_res(train_metrics, train_acc, train_ei)
        
        Y_hat, Y_hat_max = ei_model.predict(val_dataset, hp['alpha'], dataset.sensitive_attrs)
        val_acc, val_ei = model_performance(val_dataset.Y.detach().numpy(), val_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)
        append_res(val_metrics, val_acc, val_ei)
        
        Y_hat, Y_hat_max = ei_model.predict(test_dataset, hp['alpha'], dataset.sensitive_attrs)
        test_acc, test_ei = model_performance(test_dataset.Y.detach().numpy(), test_dataset.Z.detach().numpy(), Y_hat, Y_hat_max, tau)
        append_res(test_metrics, test_acc, test_ei)
    
        ei_models.append(ei_model.model)
    
    res_train = get_res(train_metrics, 'train', hp)
    res_val = get_res(val_metrics, 'val', hp)
    res_test = get_res(test_metrics, 'test', hp)
    
    
    return res_train, res_val, res_test, ei_models
    # return res_test, ei_models

In [5]:
def fb_tradeoff(dataset, hyper_params, seeds):
    hp = hyper_params.copy()
    result = pd.DataFrame()
    ei_models = []
    
    for alpha in hyper_params['alpha']:
        for lamb in hyper_params['lambda']:
            for delta in hyper_params['delta']:
                hp['alpha'] = alpha
                hp['lambda'] = lamb
                hp['delta'] = delta
                
                train, val, test, models = lr_fb_model_runner(dataset, hp, seeds)
                result = pd.concat((result, pd.DataFrame(train), pd.DataFrame(val), pd.DataFrame(test)))
                # test, models = lr_fb_model_runner(dataset, hp, seeds)
                # result = pd.concat((result, pd.DataFrame(test)))
                ei_models.extend(models)
                print()
    
    return result, ei_models

In [6]:
hyper_params = {}
hyper_params['learning_rate'] = 0.01
hyper_params['delta'] = [0.5]
hyper_params['alpha'] = [0., 0.5, 1.4]
hyper_params['lambda'] = np.linspace(0.,.25, 10).round(3)
hyper_params['z_blind'] = False
hyper_params['optimal_effort'] = False
seeds = list(range(5))

results, ei_models = fb_tradeoff(dataset, hyper_params, seeds)
results['loss_mean'] = 1 - results['accuracy_mean']
results['alpha'] = results['alpha'].astype(str)

Training [alpha=0.00; lambda=0.00]: 100%|██████████| 100/100 [00:13<00:00,  7.61epochs/s]


Training [alpha=0.00; lambda=0.03]: 100%|██████████| 100/100 [00:13<00:00,  7.28epochs/s]


Training [alpha=0.00; lambda=0.06]: 100%|██████████| 100/100 [00:13<00:00,  7.58epochs/s]


Training [alpha=0.00; lambda=0.08]: 100%|██████████| 100/100 [00:13<00:00,  7.64epochs/s]


Training [alpha=0.00; lambda=0.11]: 100%|██████████| 100/100 [00:13<00:00,  7.53epochs/s]


...

Training [alpha=1.40; lambda=0.14]: 100%|██████████| 100/100 [01:39<00:00,  1.01epochs/s]


Training [alpha=1.40; lambda=0.17]: 100%|██████████| 100/100 [01:39<00:00,  1.00epochs/s]


Training [alpha=1.40; lambda=0.19]: 100%|██████████| 100/100 [01:40<00:00,  1.01s/epochs]


Training [alpha=1.40; lambda=0.22]: 100%|██████████| 100/100 [01:41<00:00,  1.02s/epochs]


Training [alpha=1.40; lambda=0.25]: 100%|██████████| 100/100 [01:42<00:00,  1.03s/epochs]


In [7]:
results.head()

,id,alpha,lambda,delta,accuracy_mean,accuracy_std,ei_disparity_mean,ei_disparity_std,loss_mean
0,train,0.0,0.000,0.5,0.784719,0.001884,0.131939,0.004601,0.215281
0,val,0.0,0.000,0.5,0.790250,0.004839,0.130112,0.010323,0.209750
0,test,0.0,0.000,0.5,0.786450,0.002517,0.132075,0.013731,0.213550
0,train,0.0,0.028,0.5,0.782500,0.001573,0.088582,0.005080,0.217500
0,val,0.0,0.028,0.5,0.786375,0.005110,0.093586,0.010307,0.213625


In [8]:
train_results = results[results['id'] == 'train']
val_results = results[results['id'] == 'val']
test_results = results[results['id'] == 'test']

In [9]:
test_results = test_results.sort_values(['alpha', 'lambda'])
test_results

,id,alpha,lambda,delta,accuracy_mean,accuracy_std,ei_disparity_mean,ei_disparity_std,loss_mean
0,test,0.0,0.000,0.5,0.78645,0.002517,0.132075,0.013731,0.21355
0,test,0.0,0.028,0.5,0.78345,0.002790,0.089642,0.009594,0.21655
0,test,0.0,0.056,0.5,0.77890,0.003149,0.054335,0.007991,0.22110
0,test,0.0,0.083,0.5,0.76845,0.005474,0.017255,0.009134,0.23155
0,test,0.0,0.111,0.5,0.75915,0.007995,0.007835,0.003667,0.24085
0,test,0.0,0.139,0.5,0.75475,0.008905,0.008411,0.005157,0.24525
0,test,0.0,0.167,0.5,0.75225,0.008782,0.011477,0.004302,0.24775
0,test,0.0,0.194,0.5,0.74960,0.008159,0.013962,0.003467,0.25040
0,test,0.0,0.222,0.5,0.74635,0.008470,0.015571,0.004370,0.25365
0,test,0.0,0.250,0.5,0.74565,0.008524,0.017088,0.004094,0.25435


In [10]:
test_results_pareto = pd.DataFrame()
for alpha in test_results['alpha'].unique():
    test_results_alpha = test_results[test_results['alpha'] == alpha]
    mask = pareto_frontier(test_results_alpha['loss_mean'], test_results_alpha['ei_disparity_mean'])
    results_alpha_pareto = test_results_alpha.iloc[mask]
    test_results_pareto = pd.concat((test_results_pareto, results_alpha_pareto.sort_values('ei_disparity_mean')))

In [24]:
fig = px.line(test_results_pareto, x='ei_disparity_mean', y='loss_mean', color='alpha', hover_data='lambda', markers=True)
fig.add_annotation(dict(font=dict(color='black',size=10),
                                        x=0.9,
                                        y=0.99,
                                        showarrow=False,
                                        text='dataset=synthetic',
                                        textangle=0,
                                        xanchor='left',
                                        xref="paper",
                                        yref="paper"))
fig.update_layout(title=dict(text='Fairness vs Loss Tradeoff', x=0.5))
# fig.update_traces(marker=dict(size=3))
fig.show()